### Hourly forecasting of energy meter readings on BDG2 dataset

- historical data = 1 week (168 data points)
- forecast horizon = 1 day (24 data points)

**Loading TimesFM Model**

In [1]:
import os
import glob
import time
from datetime import datetime
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import islice

from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

import warnings
warnings.filterwarnings('ignore') 

In [2]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      #examples["country"].append(country)
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      #examples["gen_forecast"].append(sub_df["gen_forecast"][start:context_end + horizon_len].tolist())
      #examples["week_day"].append(sub_df["week_day"][start:context_end + horizon_len].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples

In [ ]:
# Benchmark
batch_size = 32
context_len = 168
horizon_len = 24

def process_building(df):
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    # print(input_data)
    
    windows_all = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'timestamp': inputs_ts, 
                                 'target': inputs})
        
        output_df = pd.DataFrame({'timestamp': outputs_ts, 
                                 'target': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['item_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        windows_all.append(combined)
        
    windows_all_df = pd.concat(windows_all)
    windows_all_df.timestamp = pd.to_datetime(windows_all_df.timestamp)
    windows_all_df.set_index('timestamp', inplace=True)

    return windows_all_df

In [ ]:
def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])
    df.index = pd.to_datetime(df.index)
    df['month'] = df.index.month
    training_set = df[df.month <= 6]
    test_set = df[df.month > 6]
    training_set = training_set.drop(columns='month')
    test_set = test_set.drop(columns='month')
    df = df.drop(columns='month')
    

    print(f'fine-tune set date range: {training_set.index[0]} {training_set.index[-1]}, '
      f'test set date range: {test_set.index[0]} {test_set.index[-1]}')
            

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    results_all = []
    c =0
    lag = 168 
    for building_name in df.columns:
        print(f'{datetime.now()} {c} / {len(df.columns)} {building_name}', flush=True)

        windowed_df_train = process_building(training_set[[building_name]])
        windowed_df_test = process_building(test_set[[building_name]])

        forecaster = ForecasterAutoreg(
                    regressor        = LGBMRegressor(max_depth=-1, n_estimators=100, n_jobs=24, verbose=-1),
                    lags             = 168
                )
        forecaster.fit(y= windowed_df_train['target'],
            )

        p = []
        for i in windowed_df_test.item_id_no.unique():#(pred_days):
            # i -= 1           
            seq_ptr =lag + 24 * i
        
            df_test = windowed_df_test[windowed_df_test.item_id_no == i]
            last_window  = df_test.iloc[0:168]
            ground_truth = df_test.iloc[168:192]
        
            predictions = forecaster.predict(
                steps       = 24,
                last_window = last_window['target']
            )
            # p.append(predictions)
            res = ground_truth.copy()
            res = res[['target']]
            # print(res)
            res.columns = ['y_true']
            res = res.reset_index()
            res.insert(2, 'y_pred', predictions.reset_index()['pred'])
            res.set_index('timestamp', inplace=True)
            # res['y_pred'] = predictions
            p.append(res)
        res = pd.concat(p)
        res['building'] = building_name
        results_all.append(res)
        c+=1
        # if i == 2:
        #    break
        #break
        
    results_all_df = pd.concat(results_all)
    return results_all_df

In [5]:
files_list = glob.glob('/home/user/New_Buildings_Datasets/Enernoc/csv-only/processed/*.csv')

dataset = 'Enernoc'
os.makedirs(f'forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'results/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
    if results is not None:
        results.to_csv(f'forecasts/{dataset}/{os.path.basename(filename)}')
    print('')

2024-11-13 16:32:02.147801 /home/user/New_Buildings_Datasets/Enernoc/csv-only/processed/enernoc.csv
fine-tune set date range: 2012-01-01 00:00:00 2013-01-01 00:00:00, test set date range: 2012-07-01 00:00:00 2012-12-31 23:00:00
2024-11-13 16:32:02.275803 (8785, 100)
2024-11-13 16:32:02.276937 0 / 100 767
2024-11-13 16:32:05.749490 1 / 100 304
2024-11-13 16:32:09.310452 2 / 100 399
2024-11-13 16:32:12.753437 3 / 100 21
2024-11-13 16:32:16.263183 4 / 100 805
2024-11-13 16:32:19.787510 5 / 100 14
2024-11-13 16:32:23.218720 6 / 100 404
2024-11-13 16:32:26.731603 7 / 100 78
2024-11-13 16:32:29.580464 8 / 100 731
2024-11-13 16:32:32.308202 9 / 100 218
2024-11-13 16:32:35.674850 10 / 100 366
2024-11-13 16:32:38.594048 11 / 100 766
2024-11-13 16:32:42.041079 12 / 100 197
2024-11-13 16:32:44.764247 13 / 100 30
2024-11-13 16:32:48.220647 14 / 100 742
2024-11-13 16:32:51.694169 15 / 100 32
2024-11-13 16:32:55.219824 16 / 100 137
2024-11-13 16:32:58.739036 17 / 100 36
2024-11-13 16:33:02.193906 18

### Metrics

In [6]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import root_mean_squared_log_error
from permetrics.regression import RegressionMetric

dataset = 'Enernoc'
files_list = glob.glob(f'forecasts/{dataset}/*.csv')

metrics_all_files = []

for filename in files_list:
    print(filename)
    res = pd.read_csv(filename)
    metrics_all = []
    for (g, data) in res.groupby(['building']):
        data = data.dropna()
        data = data[data.y_pred >= 0]
        print(g[0]) 
        # print(data)
        if not data.empty:
            rmse= root_mean_squared_error(data.y_true, data.y_pred)
            mae= mean_absolute_error(data.y_true, data.y_pred)
            mape = mean_absolute_percentage_error(data.y_true, data.y_pred)
            mse= mean_squared_error(data.y_true, data.y_pred)
            msle= mean_squared_log_error(data.y_true, data.y_pred)
            rmsle= root_mean_squared_log_error(data.y_true, data.y_pred)
            nrmse = rmse / (data.y_true.mean()) 
    
            evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
            nrmse_eve = evaluator.normalized_root_mean_square_error()
            evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
            smape= evaluator.symmetric_mean_absolute_percentage_error()
        
            metrics = pd.DataFrame({'building_name': [g[0]], 
                               'mae': [mae],
                                'mape': [mape],
                               'mse': [mse], 'rmse': [rmse], 'msle': [msle], 'rmsle': [rmsle], 'nrmse' : [nrmse],
                                  'nrmse_eve':[nrmse_eve] , 'sMAPE' : [smape]})
            metrics_all.append(metrics)
        else:
            continue
    
    metrics_all_df = pd.concat(metrics_all)
    metrics_all_df.to_csv(f'results/{dataset}/{os.path.basename(filename)}')

    metrics_all_df['filename'] = os.path.basename(filename)
    metrics_all_files.append(metrics_all_df)

metrics_all_files_df = pd.concat(metrics_all_files)

forecasts/Enernoc/enernoc.csv
6
8
9
10
12
13
14
21
22
25
29
30
31
32
36
41
42
44
45
49
51
55
56
65
78
88
92
99
100
101
103
109
111
116
136
137
144
153
186
197
213
214
217
218
224
228
236
259
270
275
281
285
304
339
341
363
366
384
386
391
399
400
401
404
427
454
455
472
474
475
478
484
492
496
512
648
654
673
674
690
697
703
716
718
731
737
742
744
745
755
761
765
766
767
771
786
805
808
832
887


In [7]:
metrics_all_files_df.to_csv(f'results/{dataset}/results_combined.csv')
metrics_all_files_df

,building_name,mae,mape,mse,rmse,msle,rmsle,nrmse,nrmse_eve,sMAPE,filename
0,6,34.235316,1.000412e-01,2775.531745,52.683316,0.024105,0.155257,0.143972,0.366890,0.050289,enernoc.csv
0,8,270.319820,4.420093e-01,253025.502226,503.016404,0.336168,0.579800,0.492431,0.689638,0.140912,enernoc.csv
0,9,44.392323,3.064490e+16,7092.035233,84.214222,0.771710,0.878470,0.265333,0.562788,0.076136,enernoc.csv
0,10,348.220021,1.155609e-01,319198.241515,564.976319,0.037364,0.193297,0.145595,0.689367,0.050960,enernoc.csv
0,12,28.653059,6.222646e-02,2745.781349,52.400204,0.010517,0.102554,0.111389,0.308611,0.029767,enernoc.csv
...,...,...,...,...,...,...,...,...,...,...,...
0,786,276.220499,4.657220e-01,402466.609327,634.402561,0.314648,0.560935,0.227977,1.521858,0.071253,enernoc.csv
0,805,23.405463,3.437772e-01,1404.364213,37.474848,0.212448,0.460920,0.320768,0.672066,0.122638,enernoc.csv
0,808,19.986043,2.393867e-01,1422.306115,37.713474,0.118092,0.343645,0.469585,0.536725,0.105539,enernoc.csv
0,832,233.049342,4.387444e-01,187996.822395,433.586003,0.329101,0.573674,0.490287,0.685439,0.139342,enernoc.csv


In [8]:
metrics_all_files_df.describe()*100

,building_name,mae,mape,mse,rmse,msle,rmsle,nrmse,nrmse_eve,sMAPE
count,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,33888.000000,8856.047335,3.370743e+17,6.030449e+06,14662.673957,28.346260,38.301172,28.895620,63.611739,9.465566
std,27637.111925,12343.893563,1.193034e+18,1.221486e+07,19798.246263,56.337117,37.168008,21.888849,26.171439,9.018414
min,600.000000,521.048218,4.477964e+00,5.268915e+03,725.872917,0.358495,5.987443,5.762113,30.752397,2.217663
25%,8550.000000,1704.172469,8.722212e+00,8.766004e+04,2960.726501,1.466752,12.107919,11.538959,45.469610,3.903978
50%,27800.000000,2997.935420,1.949732e+01,2.249200e+05,4742.237812,6.777003,26.030502,25.641666,57.197346,7.751731
75%,54600.000000,10502.720807,4.479374e+01,3.166099e+06,17770.536023,20.382755,45.143946,39.717644,70.067401,10.416133
max,88700.000000,50529.425142,7.014244e+18,4.984908e+07,70603.881369,306.665063,175.118549,114.995455,181.753376,54.330470
